This script takes a picture file and creates an Excel containing the RGB values (potentially scaled) of that picture ordered 
appropriately. Below the function call there is a macro which when called in the Excel file that the function creates will 
conditionally format the cells based on the input RGB values and resize the rows/columns that contain the picture to create a
more vivid/correctly scaled picture.

In [7]:
import pandas as pd
import numpy as np
import math as math
from PIL import Image

In [20]:
def pic_to_excel(picture_location,output_location,scaled = 1,width_to_scale = 128):
    #get the picture and dimensions
    im = Image.open(picture_location)
    width, height = im.size
    pix = np.asarray(im)
    
    #determine the rescaling ratio (which could be 1 if rescaling is not desired), desired dimensions and then rescale the
    #picture accordingly
    width_to_scale = max((scaled * width_to_scale), ((1-scaled) * width))
    ratio = width_to_scale / width
    scaled_width = math.floor(width * ratio)
    scaled_height = math.floor(height * ratio)
    im = im.resize((scaled_width,scaled_height), Image.ANTIALIAS)
    
    #get the scaled RGB values as an array, I believe this array is called a 3D array since it holds rows of arrays that
    #are themselves made up many arrays, each holding the 3 RGB values for that pixed
    pix = np.asarray(im)
    
    #we need to 'unpack' the 3D array from above to get a Pandas DataFrame that has one RGB value per row/column combo instead
    #of three. We create this DataFrame and then loop through each row of the array three times adding first the red the the
    #green then the blue values to the DataFrame
    for b in range(scaled_height):
        for i in range(3):
            if(b == 0 & i == 0):
                data = pd.DataFrame(pix[b,:,i].reshape((1,scaled_width)))
            else:
                data = data.append(pd.DataFrame(pix[b,:,i].reshape((1,scaled_width))),ignore_index=True)
    
    #The DataFrame is then output to the desired location
    data.to_excel(output_location,index=False,header=False)

In [21]:
pic_to_excel(picture_location = "C:\\Users\\kmerrill\\Desktop\\dog.jpg"\
             ,output_location = "C:\\Users\\kmerrill\\Desktop\\Alaina.xlsx")

Below is the VBA macro that when run will conditionally format the cells in the Excel created by the 'pic_to_excel' function as
well as adjust the size of cells and zoom out to 10% for easier viewing. This macro must be copied and then manually added to 
the Excel file. I am sure there are ways to have this happen automatically which I might add later.

-----START COPYING MACRO IMMEDIATELY BELOW THIS LINE-----

In [ ]:
'This macro is made to conditionally format cells according to their value in order to create a picture based on RGB values


Sub MakePic()
 
With Sheets("Sheet1")
'First get the last row and column with values so that only those rows and columns are altered
    LastRow = .Range("E" & .Rows.Count).End(xlUp).Row
    LastCol = .Cells(1, .Columns.Count).End(xlToLeft).Column
End With
 
 'Loop through each row that has data, conditionally format the cells Red, Green or Blue depending on row number and with 
'intensity based on cell value and set
 'the row height to be 3
    For Each rw In Worksheets("Sheet1").Rows
        If rw.Row Mod 3 = 1 And rw.Row <= LastRow Then
          rw.RowHeight = 3
          rw.FormatConditions.AddColorScale ColorScaleType:=2
          rw.FormatConditions(rw.FormatConditions.Count).SetFirstPriority
          rw.FormatConditions(1).ColorScaleCriteria(1).Type = _
              xlConditionValueNumber
          rw.FormatConditions(1).ColorScaleCriteria(1).Value = 0
          With rw.FormatConditions(1).ColorScaleCriteria(1).FormatColor
              .Color = 0
              .TintAndShade = 0
          End With
          rw.FormatConditions(1).ColorScaleCriteria(2).Type = _
              xlConditionValueHighestValue
          With rw.FormatConditions(1).ColorScaleCriteria(2).FormatColor
              .Color = 255
              .TintAndShade = 0
          End With
          End If
         
          If rw.Row Mod 3 = 2 And rw.Row <= LastRow Then
          rw.RowHeight = 3
          rw.FormatConditions.AddColorScale ColorScaleType:=2
          rw.FormatConditions(rw.FormatConditions.Count).SetFirstPriority
          rw.FormatConditions(1).ColorScaleCriteria(1).Type = _
              xlConditionValueNumber
          rw.FormatConditions(1).ColorScaleCriteria(1).Value = 0
          With rw.FormatConditions(1).ColorScaleCriteria(1).FormatColor
              .Color = 0
              .TintAndShade = 0
          End With
          rw.FormatConditions(1).ColorScaleCriteria(2).Type = _
              xlConditionValueHighestValue
          With rw.FormatConditions(1).ColorScaleCriteria(2).FormatColor
              .Color = RGB(0, 255, 0)
              .TintAndShade = 0
          End With
          End If
         
          If rw.Row Mod 3 = 0 And rw.Row <= LastRow Then
          rw.RowHeight = 3
          rw.FormatConditions.AddColorScale ColorScaleType:=2
          rw.FormatConditions(rw.FormatConditions.Count).SetFirstPriority
          rw.FormatConditions(1).ColorScaleCriteria(1).Type = _
              xlConditionValueNumber
          rw.FormatConditions(1).ColorScaleCriteria(1).Value = 0
          With rw.FormatConditions(1).ColorScaleCriteria(1).FormatColor
              .Color = 0
              .TintAndShade = 0
          End With
          rw.FormatConditions(1).ColorScaleCriteria(2).Type = _
              xlConditionValueHighestValue
          With rw.FormatConditions(1).ColorScaleCriteria(2).FormatColor
              .Color = RGB(0, 0, 255)
              .TintAndShade = 0
          End With
          End If
         
        If rw.Row > LastRow Then Exit For
    Next rw
    'Loop through each column with data and set the width to be 3
    For Each col In Worksheets("Sheet1").Columns
        If col.Column > LastCol Then Exit For
        col.ColumnWidth = 3
    Next col
   
   'Zoom the window to 10 to better see the picture
   ActiveWindow.Zoom = 10
       
End Sub
